In [2]:
import geopandas as gpd
import pandas as pd
import os

### If joining a single shapefile that contains data over multiple days, use the next two cells to load the file.

### Otherwise, skip to the multi file section

#### Load Shapefile - Single File

In [ ]:
# cd to shapefile directory 
# shp_filepath = '45.shp'
# shp_df = gpd.read_file(shp_filepath)

### Multi File (skip next two cells if single file is used)

### Use this section if there are multiple gpx shapefiles for a single device 

#### Load Shapefile - Multi File

In [110]:
# Change to directory of shapefiles
%cd '~/Documents/proj/grid3/maniema/maniema_gps/Zs Tunda'

/Users/peycke/Documents/proj/grid3/maniema/maniema_gps/Zs Tunda


In [111]:
cwd = os.path.abspath('') 
files = os.listdir(cwd)  
df = pd.DataFrame()
total_count = 0 # Total count should be the number of waypoint files
for file in files:
    if file.endswith('.shp'):
        if total_count == 0:
            shp_df = gpd.read_file(file)
            total_count +=1
        else:
            shp_df = shp_df.append(gpd.read_file(file), ignore_index=True)
            total_count+=1

print(f'There are {total_count} waypoint files in the dataframe')
print(f'The shapefile dataframe shape is {shp_df.shape}')

There are 5 waypoint files in the dataframe
The shapefile dataframe shape is (45, 9)


### Shapefile Processing

In [112]:
shp_df = shp_df.rename(columns={"Name": "NWAYPT"})
shp_df['NWAYPT'] = shp_df['NWAYPT'].astype(int)
shp_df['Type'].value_counts()

WPT    45
Name: Type, dtype: int64

### Load Excel data

In [114]:
%cd '~/Documents/proj/grid3/maniema/arranged_data'
excel_path = 'GPS04.xlsx'


excel_df = pd.read_excel(excel_path)
excel_df['NWAYPT'] = excel_df['NWAYPT'].astype(int)
df = pd.merge(excel_df, shp_df, on=['NWAYPT'])
#Verify dates are the same
print(f'Excel date is {df.Date[0]}')
print(f'GPS date is {df.DateTimeS[0]}')

/Users/peycke/Documents/proj/grid3/maniema/arranged_data
Excel date is 28/12/2018
GPS date is 2018-12-05T18:49:41Z


In [115]:
excel_wypts = excel_df['NWAYPT'].nunique()
shp_wypts = shp_df['NWAYPT'].nunique()
print(f'Excel unique waypoints:{excel_wypts} ')
print(f'GPS unique waypoints: {shp_wypts}')
# Because of duplicate entries, n unique excel waypoints will regularly be > n unique GPS waypoints

Excel unique waypoints:42 
GPS unique waypoints: 45


In [116]:
df = df.drop(columns=['Id', 'Descript','Type', 'Symbol', 'Comment']) # Drop empty or duplicate columns
df.rename(columns={'Longitude': 'Longitude_excel', 'Latitude': 'Latitude_excel','DateTimeS':'DateTime'}, inplace=True)
df.rename(columns={'lon_dd': 'Longitude', 'lat_dd': 'Latitude','DateTimeS':'DateTime'}, inplace=True)
df.set_index('NWAYPT', inplace=True)

In [117]:
df.columns = map(str.upper, df.columns)

In [118]:
%cd '~/Documents/proj/grid3/data/SNIS/processed/maniema/joined'
df.to_excel(excel_path)  

/Users/peycke/Documents/proj/grid3/data/SNIS/processed/maniema/joined


### Notes

In [ ]:
# Old Helper function for dd to dms
# def decdeg2dms(dd):
#    is_positive = dd >= 0
#    dd = abs(dd)
#    minutes,seconds = divmod(dd*3600,60)
#    degrees,minutes = divmod(minutes,60)
#    degrees = degrees if is_positive else -degrees
#    return (degrees,minutes,seconds)

# type(df['lat_dms'][0][0])
# degree_sign= u'\N{DEGREE SIGN}'
# def dms_format(dms_tuple):
#     dms_string = str(int(dms_tuple[0])) + degree_sign + str(int(dms_tuple[1]))
#     return dms_string
# lat_string = dms_format(df['lat_dms'][0])
# long_string = dms_format(df['lon_dms'][0])
# print(f'{lat_string} {long_string}')